<a href="https://colab.research.google.com/github/sovikapoudel/Data-Visualiation-Spring-2022/blob/main/Quiz_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>Data Visualization Spring 2022</center></h1>

---

###Assignment: 5
Name: Sovika Poudel <br>
Year: Junior <br>
Course: CPSMA-4313 Data Processing Visualization 
### For this part of Assignment I am going to load the libraries, gather dataset and load them here in this notebook and copy all my work from previous assignment for the better and easier reference.

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


Gathering Data

In [ ]:
%%bigquery --project data-vis-assignment-338801
SELECT *
  From `bigquery-public-data.austin_bikeshare.bikeshare_trips`
  lIMIT 5



,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900289692,Walk Up,248,2015-10-02 21:12:01+00:00,1006,Zilker Park West,1008,Nueces @ 3rd,39
1,9900285987,24-Hour Kiosk (Austin B-cycle),446,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
2,9900285989,24-Hour Kiosk (Austin B-cycle),203,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
3,9900285991,24-Hour Kiosk (Austin B-cycle),101,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,30
4,9900286140,24-Hour Kiosk (Austin B-cycle),242,2014-10-26 18:12:00+00:00,2541,State Capitol @ 14th & Colorado,2541,State Capitol @ 14th & Colorado,19


What was the second most popular starting station?

In [ ]:
%%bigquery --project data-vis-assignment-338801 
SELECT start_station_name, COUNT(start_station_id) as total_starting_point
  From `bigquery-public-data.austin_bikeshare.bikeshare_trips`
  GROUP BY start_station_name
  Order BY total_starting_point DESC ###this will give the most popular on the top so we can see 2nd most popular in 2nd row
  LIMIT 5

,start_station_name,total_starting_point
0,21st & Speedway @PCL,72799
1,Riverside @ S. Lamar,40635
2,City Hall / Lavaca & 2nd,36520
3,2nd & Congress,35307
4,Rainey St @ Cummings,34758


So the second most popular starting station is Riverside @ S. Lamar	with 40635 counts

How many trips lasted over an hour and were a round trip (started and stopped at the same station)?

At first, for this part I am just showing the list in order and confirming if the query is running correctly.

In [ ]:
%%bigquery --project data-vis-assignment-338801 
SELECT start_station_name, end_station_name, duration_minutes as time_travelled
  From `bigquery-public-data.austin_bikeshare.bikeshare_trips`
  WHERE  start_station_name = end_station_name and  duration_minutes >60
  LIMIT 100

,start_station_name,end_station_name,time_travelled
0,Toomey Rd @ South Lamar,Toomey Rd @ South Lamar,82
1,Waller & 6th St.,Waller & 6th St.,76
2,Waller & 6th St.,Waller & 6th St.,75
3,Waller & 6th St.,Waller & 6th St.,74
4,Waller & 6th St.,Waller & 6th St.,73
...,...,...,...
95,Toomey Rd @ South Lamar,Toomey Rd @ South Lamar,88
96,Toomey Rd @ South Lamar,Toomey Rd @ South Lamar,73
97,Toomey Rd @ South Lamar,Toomey Rd @ South Lamar,72
98,Waller & 6th St.,Waller & 6th St.,61


Now, for the total numbers of loop that were over 60 minutes I am using a clause within a 'WHERE' clause

In [ ]:
%%bigquery --project data-vis-assignment-338801 
SELECT COUNT(*) as No_of_Overthehour_loops
  From `bigquery-public-data.austin_bikeshare.bikeshare_trips`
  WHERE  start_station_name = end_station_name and  duration_minutes >60
  
  # Order by time_travelled DESC ### this line of code is not working correctly on this part. so commenting this out for now.
 

,No_of_Overthehour_loops
0,54795


##Assignment 6 begins from here


###Task to do!

Examine the average trip on Austin bike share grouping by property_type column from the bikeshare_stations table. Is average time on trips starting from 'parkland' longer than from 'sidewalks'? To answer this question you must join the bikeshare_trips and bikeshare_stations on the start_station_name and the name from their respective tables.

SQL Joins

In [37]:
%%bigquery --project data-vis-assignment-338801 
SELECT *
FROM `bigquery-public-data.austin_bikeshare.bikeshare_stations`
Limit 10

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,3464,Pease Park,closed,1155 Kingsbury St,None,None,None,None,None,None,None,None,9,2021-01-04 12:00:00+00:00
1,2500,Republic Square,closed,425 W 4th Street,None,None,None,None,None,None,None,None,9,2021-01-04 12:00:00+00:00
2,2536,Waller & 6th St.,closed,602 Waller St.,None,None,None,None,None,None,None,None,3,2021-01-04 12:00:00+00:00
3,2538,Bullock Museum @ Congress & MLK,closed,1881 Congress Ave.,None,None,None,None,None,None,None,None,1,2021-01-04 12:00:00+00:00
4,2541,State Capitol @ 14th & Colorado,closed,206 W. 14th St.,None,None,None,None,None,None,None,None,1,2021-01-04 12:00:00+00:00
5,2545,ACC - Rio Grande & 12th,closed,700 W. 12th St.,None,None,None,None,None,None,None,None,9,2021-01-04 12:00:00+00:00
6,2546,ACC - West & 12th Street,closed,1231 West Ave.,None,None,None,None,None,None,None,None,9,2021-01-04 12:00:00+00:00
7,2550,Republic Square @ Guadalupe & 4th St.,closed,Presented by Austin Ventures,None,None,None,None,None,None,None,None,9,2021-01-04 12:00:00+00:00
8,2564,5th & San Marcos,closed,991 East 5th,None,None,None,None,None,None,None,None,3,2021-01-04 12:00:00+00:00
9,2576,Rainey @ River St,closed,64 Rainey St,None,None,None,None,None,None,None,None,9,2021-01-04 12:00:00+00:00


First, let's gather the data about the property type and it's count.

In [ ]:
%%bigquery --project data-vis-assignment-338801 
SELECT property_type as Type, COUNT(property_type ) as Num
FROM `bigquery-public-data.austin_bikeshare.bikeshare_stations` 
WHERE property_type != " " ##There's one blank entry in the dataset under property_type so I am using the where clause to remove that from my output
group by  property_type
LIMIT 20 #just checking if this pulls out the all the informations, we only have total of 6 entries for this query though

,Type,Num
0,parkland,6
1,sidewalk,28
2,paid_parking,19
3,nonmetered_parking,8
4,undetermined_parking,19


Now, we have got the total numbers of each property types. Let's join the datasets for our next part.

In [ ]:
%%bigquery --project data-vis-assignment-338801 
WITH t as(
SELECT *
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_station_name = "Waller & 6th St.")

SELECT table1.bikeid, SUM(CASE WHEN table1.duration_minutes < table2.duration_minutes THEN 1 ELSE 0 END) as Duration_minutes_difference
FROM t as table1 JOIN t as table2 ON table1.start_station_name = table2.start_station_name
GROUP BY table1.bikeid


,bikeid,Duration_minutes_difference
0,869,45841
1,117,39552
2,965,57903
3,983,29175
4,198,25767
...,...,...
396,340,8252
397,507,19303
398,873,10239
399,233,2937


In [ ]:
%%bigquery --project data-vis-assignment-338801
SELECT *
  From `bigquery-public-data.austin_bikeshare.bikeshare_stations`
  lIMIT 5

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,3464,Pease Park,closed,1155 Kingsbury St,None,None,None,None,None,None,None,None,9,2021-01-04 12:00:00+00:00
1,2500,Republic Square,closed,425 W 4th Street,None,None,None,None,None,None,None,None,9,2021-01-04 12:00:00+00:00
2,2536,Waller & 6th St.,closed,602 Waller St.,None,None,None,None,None,None,None,None,3,2021-01-04 12:00:00+00:00
3,2538,Bullock Museum @ Congress & MLK,closed,1881 Congress Ave.,None,None,None,None,None,None,None,None,1,2021-01-04 12:00:00+00:00
4,2541,State Capitol @ 14th & Colorado,closed,206 W. 14th St.,None,None,None,None,None,None,None,None,1,2021-01-04 12:00:00+00:00


In [ ]:
%%bigquery --project data-vis-assignment-338801 
WITH t as(
SELECT *
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_station_name = "Waller & 6th St.")

SELECT *
FROM t as table1 JOIN t as table2 ON table1.start_station_name = table2.start_station_name
Limit 10

,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes,trip_id_1,subscriber_type_1,bikeid_1,start_time_1,start_station_id_1,start_station_name_1,end_station_id_1,end_station_name_1,duration_minutes_1
0,9900286171,24-Hour Kiosk (Austin B-cycle),869,2014-10-26 18:12:00+00:00,2536,Waller & 6th St.,2536,Waller & 6th St.,6,9900286171,24-Hour Kiosk (Austin B-cycle),869,2014-10-26 18:12:00+00:00,2536,Waller & 6th St.,2536,Waller & 6th St.,6
1,9900286540,24-Hour Kiosk (Austin B-cycle),117,2014-10-27 15:12:00+00:00,2536,Waller & 6th St.,2536,Waller & 6th St.,12,9900286171,24-Hour Kiosk (Austin B-cycle),869,2014-10-26 18:12:00+00:00,2536,Waller & 6th St.,2536,Waller & 6th St.,6
2,9900287861,24-Hour Kiosk (Austin B-cycle),965,2014-10-30 14:12:00+00:00,2536,Waller & 6th St.,2536,Waller & 6th St.,59,9900286171,24-Hour Kiosk (Austin B-cycle),869,2014-10-26 18:12:00+00:00,2536,Waller & 6th St.,2536,Waller & 6th St.,6
3,9900287864,24-Hour Kiosk (Austin B-cycle),983,2014-10-30 14:12:00+00:00,2536,Waller & 6th St.,2536,Waller & 6th St.,58,9900286171,24-Hour Kiosk (Austin B-cycle),869,2014-10-26 18:12:00+00:00,2536,Waller & 6th St.,2536,Waller & 6th St.,6
4,9900288274,24-Hour Kiosk (Austin B-cycle),198,2014-10-31 14:12:00+00:00,2536,Waller & 6th St.,2536,Waller & 6th St.,76,9900286171,24-Hour Kiosk (Austin B-cycle),869,2014-10-26 18:12:00+00:00,2536,Waller & 6th St.,2536,Waller & 6th St.,6
5,9900288276,24-Hour Kiosk (Austin B-cycle),327,2014-10-31 14:12:00+00:00,2536,Waller & 6th St.,2536,Waller & 6th St.,75,9900286171,24-Hour Kiosk (Austin B-cycle),869,2014-10-26 18:12:00+00:00,2536,Waller & 6th St.,2536,Waller & 6th St.,6
6,9900288278,24-Hour Kiosk (Austin B-cycle),41,2014-10-31 14:12:00+00:00,2536,Waller & 6th St.,2536,Waller & 6th St.,74,9900286171,24-Hour Kiosk (Austin B-cycle),869,2014-10-26 18:12:00+00:00,2536,Waller & 6th St.,2536,Waller & 6th St.,6
7,9900288279,24-Hour Kiosk (Austin B-cycle),277,2014-10-31 14:12:00+00:00,2536,Waller & 6th St.,2536,Waller & 6th St.,73,9900286171,24-Hour Kiosk (Austin B-cycle),869,2014-10-26 18:12:00+00:00,2536,Waller & 6th St.,2536,Waller & 6th St.,6
8,9900288445,Annual Membership (Austin B-cycle),223,2014-10-31 17:12:00+00:00,2536,Waller & 6th St.,2536,Waller & 6th St.,1,9900286171,24-Hour Kiosk (Austin B-cycle),869,2014-10-26 18:12:00+00:00,2536,Waller & 6th St.,2536,Waller & 6th St.,6
9,9900288447,Annual Membership (Austin B-cycle),223,2014-10-31 17:12:00+00:00,2536,Waller & 6th St.,2536,Waller & 6th St.,14,9900286171,24-Hour Kiosk (Austin B-cycle),869,2014-10-26 18:12:00+00:00,2536,Waller & 6th St.,2536,Waller & 6th St.,6
